## Traditional machine learning
Feature(s): smiles_nodes,smiles_linkers
- translate to morgan fingerprint

target: HER_bool,OER_bool,VIS_bool
- three binary classifications

In [1]:
import sys
import os
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="WARNING")
# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

Read the dataset from the zip file

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'MOFs_photocatalysis.csv'
csv_filename = 'MOFs_photocatalysis.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)




In [3]:
df.head()

,mofid,mofkey,smiles_nodes,smiles_linkers,smiles,topology,cifname,HER_bool,OER_bool,VIS_bool,four_classes,inorg_BB,inorg_BB_top,hased_linker,HOV_bool,HOV_code,elements_linkers,elements_nodes,elemetns_all
0,[O-]C(=O)c1ccc(c(c1)N)C(=O)[O-].[Ti]12[O]3[Ti]...,Ti.GPNNOCMCNFXRAO.MOFkey-v1.hxl,['[Ti]12[O]3[Ti]4[O]2[Ti]2[O]4[Ti]4[O]5[Ti]3[O...,['[O-]C(=O)c1ccc(c(c1)N)C(=O)[O-]'],[O-]C(=O)c1ccc(c(c1)N)C(=O)[O-].[Ti]12[O]3[Ti]...,hxl,hxl_v1-MOF901_1-L8_opt,0,0,0,3,Ti,Ti and hxl,GPNNOCMCNFXRAO,0,0,"['O', 'C', 'N']","['O', 'Ti']","['O', 'C', 'N', 'Ti']"
1,Oc1c(c2ccc(s2)C(=O)[O-])c2nc3ccccc3nc2c(c1O)c1...,Ti.NMIBMOGMZWZBEY.MOFkey-v1.hxl,['[Ti]12[O]3[Ti]4[O]2[Ti]2[O]4[Ti]4[O]5[Ti]3[O...,['Oc1c(c2ccc(s2)C(=O)[O-])c2nc3ccccc3nc2c(c1O)...,Oc1c(c2ccc(s2)C(=O)[O-])c2nc3ccccc3nc2c(c1O)c1...,hxl,hxl_v1-MOF901_1-link25opt,1,1,1,0,Ti,Ti and hxl,NMIBMOGMZWZBEY,111,1,"['O', 'C', 'Oc']","['O', 'Ti']","['O', 'C', 'Oc', 'Ti']"
2,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,TiZr.SJFHFFGLOPKJGJ.ZEOFITWTNNBMQJ.MOFkey-v1.E...,['O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti...,['[O-]C(=O)c1sc(c(c1)C)c1sc(c2c1nccn2)c1sc(cc1...,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,ERROR,Ti8Zr2COO12_link24opt_clean_maestro2,1,0,1,1,TiZr,TiZr and ERROR,SJFHFFGLOPKJGJ,101,2,"['O', 'C']","['O', 'Zr', 'Ti']","['O', 'C', 'Ti', 'Zr']"
3,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,TiZr.NMIBMOGMZWZBEY.MOFkey-v1.ERROR,['O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti...,['Oc1c(c2ccc(s2)C(=O)[O-])c2nc3ccccc3nc2c(c1O)...,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,ERROR,Ti8Zr2COO12_kink25opt,1,1,1,0,TiZr,TiZr and ERROR,NMIBMOGMZWZBEY,111,1,"['O', 'C', 'Oc']","['O', 'Zr', 'Ti']","['C', 'Zr', 'O', 'Oc', 'Ti']"
4,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,TiZr.BQYNSOHOKMNSBV.MOFkey-v1.ERROR,['O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti...,['[O-]C(=O)c1cc2n(c1C)[B](F)(F)n1c([C]2c2ccccc...,O1[Ti]2[O]3[Ti]4O[Ti]5[O]6[Ti]1[O]1[Ti]7O[Ti]8...,ERROR,Ti8Zr2COO12_link15opt,1,1,1,0,TiZr,TiZr and ERROR,BQYNSOHOKMNSBV,111,1,"['O', 'C', 'B', 'F']","['O', 'Zr', 'Ti']","['C', 'F', 'Zr', 'O', 'Ti', 'B']"


In [4]:
from MLPipeline import MLmodel, BinTheTarget


Target = ['HER_bool','OER_bool','VIS_bool']
Features = ['smiles_nodes','smiles_linkers']
Feature_types = ['SMILES']*len(Features)
input = df

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def objectiveRandomForestClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


def objectiveXGBClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize for XGBClassifier.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),  # L2 regularization
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


In [6]:
MODEL_NAMES = ['RandomForestClassifier', 'XGBClassifier']
TARGETS = [[i] for i in Target ]
TRAIN_SIZES = [25, 50, 100, 200]

result = []
for seed in [1, 2, 3]:
    for model_name in MODEL_NAMES:
        for target in TARGETS:
            for train_size in TRAIN_SIZES:
                print(f'RUN: Model:{model_name} / Target:{target} / Train size:{train_size} / Seed:{seed}')
                
                if model_name == 'RandomForestClassifier':
                    objective = objectiveRandomForestClassifier
                elif model_name == 'XGBClassifier':
                    objective = objectiveXGBClassifier

                model = MLmodel(modelType=model_name, 
                df=input,
                randomSeed=seed,
                train_count = train_size,
                test_count = 50, 
                target=target, 
                features=Features, 
                hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', 
                optimization_trials=2,
                objective=lambda trial: objective(trial, model)
                )

                model.train()
                eval, summary = model.evaluate()
                result.append({**summary, **eval, 'seed':seed})



[15:03:56] SMILES Parse Error: syntax error while parsing: ['[Cu]1[Cu]2[Cu]31[Cu]1[Cu]2[Cu]31',
[15:03:56] SMILES Parse Error: Failed parsing SMILES '['[Cu]1[Cu]2[Cu]31[Cu]1[Cu]2[Cu]31',' for input: '['[Cu]1[Cu]2[Cu]31[Cu]1[Cu]2[Cu]31','
[15:03:56] SMILES Parse Error: syntax error while parsing: ['[Cu]']
[15:03:56] SMILES Parse Error: Failed parsing SMILES '['[Cu]']' for input: '['[Cu]']'
[15:03:56] SMILES Parse Error: syntax error while parsing: ['[Cu]',
[15:03:56] SMILES Parse Error: Failed parsing SMILES '['[Cu]',' for input: '['[Cu]','
[15:03:56] SMILES Parse Error: syntax error while parsing: ['[Ti]12[O]3[Ti]4[O]2[Ti]2[O]4[Ti]4[O]5[Ti]3[O]1[Ti]5[O]24']
[15:03:56] SMILES Parse Error: Failed parsing SMILES '['[Ti]12[O]3[Ti]4[O]2[Ti]2[O]4[Ti]4[O]5[Ti]3[O]1[Ti]5[O]24']' for input: '['[Ti]12[O]3[Ti]4[O]2[Ti]2[O]4[Ti]4[O]5[Ti]3[O]1[Ti]5[O]24']'
[15:03:56] SMILES Parse Error: syntax error while parsing: ['[Ag]',
[15:03:56] SMILES Parse Error: Failed parsing SMILES '['[Ag]',' for input: '

RUN: Model:RandomForestClassifier / Target:['HER_bool'] / Train size:25 / Seed:1


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/lsmo/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/lsmo/.local/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/lsmo/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "/home/lsmo/.local/lib/python3.10/site-packages/sklearn/base.py", line 622, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/home/lsmo/.local/lib/python3.10/site-packag

Some information about the dataset such as the splitting, wrong smiles, total number of samples.

In [ ]:
df = pd.DataFrame(result)
display(df)
df.to_csv('Photocat_waterSplitting_MOFs_traditional_ml.csv')

In [ ]:
import matplotlib.pyplot as plt

METRICS = ['accuracy','f1_micro','f1_macro', 'kappa']
fig, ax = plt.subplots(len(METRICS), 1, sharex=True, layout = 'constrained')
ax = ax.flatten()

for i, metric in enumerate(METRICS):
    ax[i].plot(df['train_size'], df[metric], marker = 'o')
    ax[i].set_title(metric)
    ax[i].set_ylim(0,1)

fig.suptitle('Random Forrest & XGB - Photocat_waterSplitting_MOFs')